In [ ]:
import requests
import re
import numpy as np
from bs4 import BeautifulSoup as BS
from requests.auth import HTTPBasicAuth

- **requests**

In [ ]:
def getresp(url,params,author):
    resp = requests.get(url,params=params,auth=author)
    if resp.status_code == requests.codes.ok:
        print('Visiting url successful!')
        pass
    else:
        resp.raise_for_status()
        # quit()
    return resp

- **BeautifulSoup**

In [ ]:
def geturl(resp):
    fileurl = []
    soup = BS(resp.text,'lxml')
    entrys = soup.findChildren('entry')
    for entry in entrys:
        fileurl.append(entry.link.attrs['href'])
    return fileurl

## User Information

In [ ]:
user = 'qindiandadudu'
password = 'abcd02251214'
author = HTTPBasicAuth(user,password)

## Search URL information and parameters setting

- search url

In [ ]:
url = 'https://scihub.copernicus.eu/dhus/search'

- start number and  row number of each page

In [ ]:
start = 0
rows = 100

- search results order 

    beginposition asc : sorts results by sensing date arranged in ascending order
    
    beginposition desc : sorts results by sensing date arranged in descending order
    
    ingestiondate asc: sorts results by ingestion date arranged in ascending order
    
    ingestiondate desc: sorts results by ingestion date arranged in descending order 

In [ ]:
orderby = 'beginposition desc'

- period

In [ ]:
beginposition = '[2015-09-01T00:00:00.000Z TO 2017-11-14T23:59:59.999Z]'

In [ ]:
endposition = '[2015-09-01T00:00:00.000Z TO 2017-11-14T23:59:59.999Z]'

- filename

In [ ]:
filename = 'S2A_*'

- platformname

In [ ]:
platformname = 'Sentinel-2'

- producttype

In [ ]:
producttype = 'S2MSI1C'

- search parameters

In [ ]:
params={'q': 'footprint:"Intersects(POLYGON((-125.31488630127798 49.37923341676577,-94.1177312732597 49.611074219444475,-89.47742217298237 48.53549820848218,-88.12102412828592 48.53549820848218,-82.12431698331216 45.81890506556195,-81.33903390480367 43.37952244235831,-82.26709572485915 42.332904057015014,-79.41152089391926 42.91073956185417,-79.55429963546625 43.793199048601366,-76.91289291684686 43.84470870363458,-75.05676927673592 45.21866420479017,-71.41591136728756 45.41945598010943,-69.13145150253564 47.86937637130322,-67.27532786242472 47.3399115244641,-66.49004478391625 44.510289298419195,-68.70311527789465 43.99897102421997,-70.4878495472321 42.54364651046643,-70.63062828877908 41.26857486211881,-73.55759249049247 40.35002427915461,-75.5564948721504 37.23493437901901,-75.34232675982992 35.5106407926214,-81.26764453403018 30.91389921859202,-79.55429963546625 26.277507711359192,-80.41097208474821 24.601441451662325,-82.05292761253865 25.699976916085063,-83.40932565723509 29.246069409930726,-86.47906860049548 29.92894696853658,-89.26325406066188 29.92894696853658,-90.76243084690535 29.31312823256077,-93.47522693629823 29.31312823256077,-96.9733061041996 27.428820161771597,-96.9733061041996 25.253804342260892,-99.8288809351395 26.28243160798492,-101.32805772138293 29.188555415633374,-103.32696010304085 28.938955736789637,-104.9689156308313 29.56181780055128,-105.32586248469879 30.242555555774018,-107.5389329786772 31.40731039455048,-108.89533102337366 30.73469982619183,-113.32147201133047 31.58992129699972,-115.17759565144141 32.377092871245935,-117.39066614541981 32.377092871245935,-118.74706419011625 33.93095765190809,-120.6745772010007 34.22660464625842,-122.38792209956462 36.439439833782075,-124.52960322276952 39.97231578245231,-124.67238196431654 43.17562317292362,-124.31543511044906 47.00393239105679,-125.31488630127801 48.58637973178338,-125.31488630127798 49.37923341676577,-125.31488630127798 49.37923341676577)))"'
        + ' AND filename:'+ filename + ' AND platformname:' + platformname + ' AND producttype:' + producttype 
        + ' AND beginposition:' + beginposition + ' AND endPosition:' + endposition,
        'orderby': orderby,
        'start': start,
        'rows': rows}

# ** main program**

In [ ]:
urldatabase = []

In [ ]:
wrongpages = []

- get search results information

In [ ]:
resp = getresp(url=url,params=params,author=author)
soup = BS(resp.text,'lxml')
subtitle = soup.subtitle.contents[0]
d = re.search(r'Displaying',subtitle)
t = re.search(r'to',subtitle)
o = re.search(r'of',subtitle)
tot = re.search(r'total',subtitle)
start = int(subtitle[d.end():t.start()])
resultnumber = int(subtitle[o.end():tot.start()])

In [ ]:
resultnumber

In [ ]:
# 计算获取结果页面数--即主循环次数
if (resultnumber%rows) == 0:
    pages = resultnumber//rows
else:
    pages = resultnumber//rows + 1

In [ ]:
pages

In [ ]:
# 下一次访问时records开始数
nextstart = start

In [ ]:
# 重新启动位置
restart = start

In [ ]:
for i in range(restart,pages):
    nextstart = i*rows
    params={'q': 'footprint:"Intersects(POLYGON((87.96744928079151 49.456519812462346,91.60830719023988 46.83752798473412,91.25136033637239 45.602765307819425,95.89166943664972 44.64576303006464,96.7483418859317 43.15413083756209,101.46004035698249 42.94545696139997,104.8153407833369 42.156611918760944,109.95537547902869 42.94545696139997,111.38316289449862 43.827415120460785,111.09760541140463 45.000210707779814,112.16844597300711 45.50278424803122,113.73901213002404 45.201773800610084,118.80765745494234 47.37203214140084,114.80985269162649 47.75739353110845,116.52319759019043 50.28470084157169,117.95098500566037 50.01021802911433,120.52100235350628 52.467671174043375,119.30738305035682 52.68458961189609,121.09211731969427 53.71091470506266,123.94769215063415 53.79533483672091,125.94659453229207 53.2862648577391,127.94549691395 49.96431767943767,130.8010717448899 48.943405599629216,131.30079734030437 48.044555096620655,135.2272127328467 48.61410385272944,135.01304462052622 47.66131893588616,133.51386783428276 44.747256163082966,131.80052293571885 45.000210707779814,131.30079734030437 42.52598030538988,121.80601102742924 38.176115746490154,123.59074529676666 37.612756017996006,120.23544487041231 35.14161329563284,122.16295788129669 32.23150772949295,123.23379844289917 30.278766301259637,118.73626808416886 23.994003317793286,112.5253928268746 21.159899865927372,110.59787981599015 18.676749577533087,110.81204792831065 20.62633973748771,107.52813687272979 20.96003728051427,105.74340260339234 22.617193193148964,101.46004035698252 20.96003728051427,99.10419112145709 21.75786247555702,97.46223559366666 24.31969121888956,97.96196118908115 27.46602416787246,95.32055447046173 28.72552358988112,93.03609460570983 27.719103082020624,89.82357292090245 27.21236283607496,86.61105123609507 27.402663295197584,81.89935276504427 29.66033159380234,78.11571611404891 31.32119224098024,78.11571611404891 34.261260362316236,77.47321177708743 35.19996972135458,74.76041568769453 36.530544485875296,73.33262827222458 38.84644093997744,73.33262827222458 39.949730443191925,75.3315306538825 40.98163678406533,76.25959247393799 41.03550801313756,77.68737988940791 41.518365501224054,79.75767164183932 42.262367141059,80.04322912493333 43.569347922768316,79.68628227106583 45.000210707779814,81.75657402349728 45.602765307819425,82.68463584355271 47.420357376064516,84.68353822521065 47.37203214140084,87.96744928079151 49.456519812462346,87.96744928079151 49.456519812462346)))"'
            + ' AND filename:'+ filename + ' AND platformname:' + platformname + ' AND producttype:' + producttype 
            + ' AND beginposition:' + beginposition + ' AND endPosition:' + endposition,
            'orderby': orderby,
            'start': nextstart,
            'rows': rows}
    resp = getresp(url=url,params=params,author=author)
    urllist = geturl(resp)
    if i < pages-1:
        if len(urllist) == rows:
            urldatabase = urldatabase + urllist
            print('Page',i+1,': finished!',' ------ URL list length:',len(urllist),' ------ records numbers',len(urldatabase))
        else:
            print('abstract url fail!')
            wrongpages.append(i+1)
    else:
        if len(urllist) == (resultnumber - rows * i):
            urldatabase = urldatabase + urllist
            print('Page',i+1,': finished!',' ------ URL list length:',len(urllist),' ------ records numbers',len(urldatabase))
        else:
            print('abstract url fail!')
            wrongpages.append(i+1)

In [ ]:
wrongpages

- 查看中断页数

In [ ]:
if i < pages - 1:
    print('中断页数：',i)
else:
    pass

- 保存爬取到的url

In [ ]:
for i in range(len(urldatabase)):
    a = urldatabase[i]
    urldatabase[i] = a[0:8] + user + ':' + password + '@' + a[8::]

In [ ]:
urldatabase

In [ ]:
urldatabase_np = np.array(urldatabase)

- 单个文件纪录长度：

In [ ]:
filelength = 1000

- 文件数：

In [ ]:
filenumber = resultnumber//filelength + 1

In [ ]:
for num in range(filenumber):
    if num < filenumber - 1:
        np.savetxt('urls' + str(num*filelength+1) + '-' + str((num+1)*filelength) + '.txt',
                   urldatabase_np[num*filelength:(num+1)*filelength],fmt='%s')
    else:
        np.savetxt('urls' + str(num*filelength+1) + '-' + str(resultnumber) + '.txt',
                   urldatabase_np[num*filelength:(num+1)*filelength],fmt='%s')